In [51]:
import pandas as pd

In [52]:
PATH = "gs://induction-labs/jonathan/sampled_trajectories/osworld_uitars_5x_en_fixed_browser"
DATA_FILE = f"{PATH}/samples.jsonl"

In [53]:
# fix samples
ORIGINAL_TRAJECTORIES = pd.read_json(DATA_FILE, lines=True)
FIXED_TRAJECTORIES = ORIGINAL_TRAJECTORIES.copy()
FIXED_TRAJECTORIES["eval_task_id"] = FIXED_TRAJECTORIES["eval_task_id"].str.replace(r"-2$", "", regex=True)
FIXED_TRAJECTORIES = FIXED_TRAJECTORIES[(FIXED_TRAJECTORIES["reward"] == 0) | (FIXED_TRAJECTORIES["reward"] == 1)]

In [54]:
FIXED_TRAJECTORIES.to_json(f"{PATH}/samples_fixed.jsonl", orient="records", lines=True)

In [55]:
FIXED_TRAJECTORIES = pd.read_json(f"{PATH}/samples_fixed.jsonl", lines=True)

In [56]:
import pandas as pd
import numpy as np

def expand_with_sliding_windows(df: pd.DataFrame, width: int,
                                length_col: str = "trajectory_length") -> pd.DataFrame:
    """
    For every row in *df*, create (trajectory_length – width + 1) copies,
    one for every sliding window of size *width*.
    Adds two new columns:  `window_start` and `window_end` (inclusive).
    Rows whose `trajectory_length` < width are silently dropped.

    Parameters
    ----------
    df : pd.DataFrame                 Original data.
    width : int                       Size of the sliding window.
    length_col : str, default 'trajectory_length'
                                      Column holding the total length.

    Returns
    -------
    pd.DataFrame                      Expanded frame.
    """
    if width <= 0:
        raise ValueError("width must be a positive integer")

    # how many windows each row will produce
    n_windows = df[length_col] - width + 1

    # keep only rows where at least one window fits
    valid_mask = n_windows > 0
    df_valid   = df[valid_mask].copy()
    n_windows  = n_windows[valid_mask]

    # repeat the rows the right number of times
    expanded = df_valid.loc[df_valid.index.repeat(n_windows)].reset_index(drop=True)

    # build the window start indices in one NumPy shot
    starts = np.concatenate([np.arange(k) for k in n_windows])
    expanded["image_turns_start"] = starts
    expanded["image_turns_end"]   = starts + width        # inclusive upper bound
    expanded["text_turns_start"] = 0
    expanded["text_turns_end"]   = starts + width

    return expanded

In [57]:
FIXED_TRAJECTORIES

,eval_task_id,attempt_id,instruction,reward,output_folder,trajectory_length
0,2ad9387a-65d8-4e33-ad5b-7580065a27ca,c3daf13f1cb34ca9a735cd9e66378fd4,Can you make a new folder for me on the bookma...,0,gs://induction-labs-data-ext-mumbai/evals/test...,2
1,ce88f674-ab7a-43da-9201-468d38539e4a,4cd29b7218704f44be3606df73c9b3a1,Please set my slides upright instead of sideways.,0,gs://induction-labs-data-ext-mumbai/evals/test...,2
2,2cd43775-7085-45d8-89fa-9e35c0a915cf,373726d9a6a54ffab4c1705d11be15e6,"Enable auto-save every 3min for me, so that I ...",0,gs://induction-labs-data-ext-mumbai/evals/test...,2
3,3f28fe4f-5d9d-4994-a456-efd78cfae1a3,33192184178245709e43ade65349d05c,Set up a signature using my name and affiliati...,0,gs://induction-labs-data-ext-mumbai/evals/test...,2
4,b148e375-fe0b-4bec-90e7-38632b0d73c2,be66098c436c4fde9210af956e5370d5,Could you assist me in adding a new layer and ...,0,gs://induction-labs-data-ext-mumbai/evals/test...,2
...,...,...,...,...,...,...
1311,535364ea-05bd-46ea-9937-9f55c68507e8,55cbc75da9d048248ed8a6bcd0816683,Create two tables in a new sheet showing the t...,0,gs://induction-labs-data-ext-mumbai/evals/test...,97
1312,e2392362-125e-4f76-a2ee-524b183a3412,ba9d2bb7e815485dbf5137bc9492c028,I recently started using the famous personal a...,0,gs://induction-labs-data-ext-mumbai/evals/test...,101
1313,b52b40a5-ad70-4c53-b5b0-5650a8387052,47a8a1593c4e4bd0829411633332b9b6,"Could you help me merge all PDF files in the ""...",0,gs://induction-labs-data-ext-mumbai/evals/test...,98
1314,adf5e2c3-64c7-4644-b7b6-d2f0167927e7,723529122c004e7eae9c55d58fd063f7,"Help me adding ""Steinberg, F. M., Bearden, M. ...",0,gs://induction-labs-data-ext-mumbai/evals/test...,101


In [58]:
CORRECT_TRAJECTORIES_EXPANDED = FIXED_TRAJECTORIES.copy()
CORRECT_TRAJECTORIES_EXPANDED = CORRECT_TRAJECTORIES_EXPANDED[CORRECT_TRAJECTORIES_EXPANDED["reward"] == 1]
CORRECT_TRAJECTORIES_EXPANDED = expand_with_sliding_windows(
    CORRECT_TRAJECTORIES_EXPANDED,
    width=5,
    length_col="trajectory_length"
)
num = 20
CORRECT_TRAJECTORIES_EXPANDED = CORRECT_TRAJECTORIES_EXPANDED[CORRECT_TRAJECTORIES_EXPANDED["text_turns_end"] <= num]
CORRECT_TRAJECTORIES_EXPANDED.to_json(f"{PATH}/samples_correct_trajectories_expanded_under_{num}.jsonl", orient="records", lines=True)

In [59]:
shuffled = CORRECT_TRAJECTORIES_EXPANDED.sample(frac=1, random_state=248239)
test_size = 64
train_size = len(shuffled) - test_size
TRAIN_SET = shuffled.iloc[:train_size]
TEST_SET = shuffled.iloc[train_size:]

TRAIN_SET.to_json(f"{PATH}/samples_correct_trajectories_expanded_under_{num}_train.jsonl", orient="records", lines=True)
TEST_SET.to_json(f"{PATH}/samples_correct_trajectories_expanded_under_{num}_test.jsonl", orient="records", lines=True)